In [1]:
!pip install pulp

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import random
from itertools import permutations
from pulp import LpProblem, LpVariable, LpStatus, LpMinimize, lpSum
from typing import Dict, List, Tuple
from pulp import GLPK_CMD

def generate_dummy_data(n_pois: int, n_days: int) -> tuple:
    """
    Generate dummy data for the itinerary optimization problem.

    Parameters:
    - n_pois: The number of points of interest (POIs).
    - n_days: The number of days.

    Returns:
    - pois: A list of POI names.
    - days: A list of day identifiers.
    - travel_time: A dictionary with travel times between each pair of POIs.
    """
    pois = [f'poi_{i}' for i in range(n_pois)]
    days = [f'D{day}' for day in range(1, n_days + 1)]
    travel_time = {(poi1, poi2): random.randint(10, 60) for poi1, poi2 in permutations(pois, 2)}
    return pois, days, travel_time

In [ ]:
# Initialize data
n_pois = 5  # Number of POIs
n_days = 2  # Number of days
pois, days, travel_time = generate_dummy_data(n_pois, n_days)

In [3]:
# Initialize the problem
problem = LpProblem("Itinerary_Optimization", LpMinimize)

# Variables
order_vars = {
    (poi1, poi2, day): LpVariable(f"order_{poi1}_{poi2}_{day}", cat="Binary")
    for poi1 in pois for poi2 in pois for day in days if poi1 != poi2
}

# Objective function: Minimize the total travel time
problem += lpSum(
    travel_time[(poi_from, poi_to)] * order_vars[(poi_from, poi_to, day)]
    for poi_from, poi_to in permutations(pois, 2) for day in days
)

# Constraints

# Each POI visited exactly once across all days
for poi in pois:
    problem += lpSum(
        order_vars[(poi, other_poi, day)] + order_vars[(other_poi, poi, day)]
        for day in days for other_poi in pois if poi != other_poi
    ) == 2, f"enter_and_exit_{poi}"

# Ensure entering and exiting each POI occurs within the same day
for day in days:
    for poi in pois:
        problem += lpSum(
            order_vars[(poi, other_poi, day)]
            for other_poi in pois if poi != other_poi
        ) == lpSum(
            order_vars[(other_poi, poi, day)]
            for other_poi in pois if poi != other_poi
        ), f"same_day_visit_{poi}_{day}"

# Constraint: Ensure each day has at least 2 POI visited
for day in days:
    problem += lpSum(
        order_vars[(poi1, poi2, day)]
        for poi1 in pois for poi2 in pois if poi1 != poi2
    ) >= 2, f"at_least_one_poi_{day}"

# Constraint: Ensure that these 2 POIs are not visited on the same day
not_same_day_pois = pois[1:3]
for day in days:
    problem += lpSum(
        order_vars[(poi, other_poi, day)]
        for poi in not_same_day_pois for other_poi in pois if poi != other_poi
    ) <= 2, f"not_same_day_pois_{day}"
    problem += lpSum(
        order_vars[(poi, other_poi, day)]
        for poi in not_same_day_pois for other_poi in not_same_day_pois if poi != other_poi
    ) == 0, f"not_consec_pois_{day}"

# problem
# Solve the problem
status = problem.solve(GLPK_CMD(msg=0))

# Check the status and print the results
if LpStatus[status] == 'Optimal':
    print("Solution Found:\n")
    results = []
    for var in order_vars:
        if order_vars[var].varValue == 1:
            results.append(var)
            print(f"{var} = {order_vars[var].varValue}")
else:
    print("No optimal solution found.")


Solution Found:

('poi_0', 'poi_2', 'D2') = 1
('poi_1', 'poi_4', 'D1') = 1
('poi_2', 'poi_0', 'D2') = 1
('poi_3', 'poi_1', 'D1') = 1
('poi_4', 'poi_3', 'D1') = 1
